In [1]:
print("execution started")

import mlflow
import transformers


modelname = 'meta-llama/Llama-2-7b'
from transformers import GPT2LMHeadModel
from transformers import GPT2Tokenizer

print("before tokenizer")
tokenizer = GPT2Tokenizer.from_pretrained(modelname)
print("after tokenizer model")
model = GPT2LMHeadModel.from_pretrained(modelname)
print("after automodel")



basemodel = mlflow.transformers.log_model(
        transformers_model={"model": model, "tokenizer": tokenizer },
        task="text-generation",
        artifact_path="models/basemodel",
        input_example=["base model loaded directly from transformer's"],
        registered_model_name="Gpt-2-medium-base-model"
        )
print("registered the model")


mlflow.transformers.save_model(
    transformers_model={"model": model, "tokenizer": tokenizer },
    path="transformersmodel"
)

mlmodel = mlflow.transformers.load_model(
    "transformersmodel"
)



from datasets import load_dataset
traindataset = load_dataset("cnn_dailymail","3.0.0", split="train").shuffle().select(range(1000))
testdataset = load_dataset("cnn_dailymail", "3.0.0", split="test").shuffle().select(range(500))

def tokenizer_function(examples):
        return tokenizer(examples["article"],text_target=examples["highlights"],truncation=True,padding=True,max_length=128,return_tensors="pt")

tokenizer.pad_token = tokenizer.eos_token
traindataset = traindataset.map(tokenizer_function,batched=True)
testdataset = testdataset.map(tokenizer_function,batched=True)

tokenizer.pad_token = tokenizer.eos_token
traindataset = traindataset.map(tokenizer_function,batched=True)
testdataset = testdataset.map(tokenizer_function,batched=True)

traindataset = traindataset.remove_columns(["article","highlights","id"])
testdataset = testdataset.remove_columns(["article","highlights","id"])

import evaluate
metrics = evaluate.load("perplexity")

import numpy as np
def compute_metrics(eval_period):
        print(eval_period) 
        logits, labels = eval_period
        predictions = np.argmax(logits,x_axis = 1)
        return metrics.compute(predictions = predictions, references = labels)


import os
os.environ["DISABLE_MLFLOW_INTEGRATION"] = "TRUE"

from transformers import TrainingArguments,Trainer

Training_args = TrainingArguments(
        num_train_epochs = 1,
        output_dir= "output",
        save_strategy="epoch"
    )

trainer = Trainer(
        model = mlmodel.model,
        tokenizer=tokenizer,
        args = Training_args,
        compute_metrics=compute_metrics,
        train_dataset = traindataset,
        eval_dataset = testdataset,
        
    )

result = trainer.train()

print(result)
# trainedmodel = mlflow.transformers.log_model(
#         transformers_model={"model": trainer.model, "tokenizer": tokenizer },
#         task="text-generation",
#         artifact_path="models/trainedmodel",
#         input_example=["base model loaded directly from transformer's"],
#         registered_model_name="Gpt-2-medium-base-model"
#         )

# mlflow.end_run()

# print("trainer model got logged successfully")



execution started
before tokenizer
after tokenizer model
after automodel
registered the model
Dataset cnn_dailymail downloaded and prepared to /home/azureuser/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_12242/2363559458.py:19: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.30.2``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  basemodel = mlflow.transformers.log_model(
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/models/model.py:572: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.30.2``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  flavor.save

Step,Training Loss


Attempted to log scalar metric train_runtime:
267.4291
Attempted to log scalar metric train_samples_per_second:
3.739
Attempted to log scalar metric train_steps_per_second:
0.467
Attempted to log scalar metric total_flos:
232175173632000.0
Attempted to log scalar metric train_loss:
4.2668056640625
Attempted to log scalar metric epoch:
1.0
TrainOutput(global_step=125, training_loss=4.2668056640625, metrics={'train_runtime': 267.4291, 'train_samples_per_second': 3.739, 'train_steps_per_second': 0.467, 'total_flos': 232175173632000.0, 'train_loss': 4.2668056640625, 'epoch': 1.0})


In [2]:
model_tokenizer = {"model":model, "tokenizer":tokenizer}

In [7]:
model = model_tokenizer["model"]
tokenizer = model_tokenizer["tokenizer"]
inputs = tokenizer("Hello, my dog is ", "The movie was ", return_tensors="pt")
output = model(**inputs)
predictions = torch.nn.functional.softmax(output.logits, dim=-1)
print(f'Predicted class: {predictions}')

Predicted class: tensor([[[7.3768e-02, 3.5655e-02, 3.0460e-05,  ..., 1.1749e-09,
          8.6480e-09, 1.5610e-03],
         [4.2479e-04, 1.0622e-04, 1.2661e-05,  ..., 2.4750e-07,
          2.7505e-07, 3.5757e-04],
         [8.7142e-05, 1.8022e-05, 4.4241e-07,  ..., 3.9378e-07,
          5.4928e-08, 3.2181e-06],
         ...,
         [1.4669e-02, 4.7507e-03, 9.1801e-06,  ..., 1.1602e-08,
          3.2272e-07, 6.4735e-04],
         [2.4840e-04, 9.0125e-05, 7.3380e-06,  ..., 2.6484e-09,
          2.0246e-07, 5.6557e-05],
         [1.7301e-03, 5.5349e-03, 6.0197e-04,  ..., 8.7947e-10,
          1.1176e-07, 2.4396e-04]]], grad_fn=<SoftmaxBackward0>)
